In [1]:
trainingset_folder    = "D:/data_mt/09_training/"
stock_data_folder = trainingset_folder + "stocks/"
combine_data_folder = trainingset_folder + "combined/"

combined_data_file = trainingset_folder + "/all_combine_data"


# 01_Tools

In [2]:
import findspark
from pyspark.sql import SparkSession
def get_spark_session(appname = "default"):
    """
    Initialises a spark session. 
    Parameters:
    appname - default is "default"
    """
    findspark.init()
    return SparkSession.builder \
                        .appName(appname) \
                        .getOrCreate()

In [3]:
# init Spark
spark = get_spark_session() # Session anlegen
spark # display the moste important information of the session

# 02_Load Data

In [4]:
df_all = spark.read.parquet(combined_data_file)

In [ ]:
dfparquet